In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_data = pd.read_csv("/kaggle/input/fungi-clef-2025/metadata/FungiTastic-FewShot/FungiTastic-FewShot-Train.csv")
train_data.head()

,eventDate,year,month,day,habitat,countryCode,scientificName,kingdom,phylum,class,...,observationID,region,district,filename,category_id,metaSubstrate,poisonous,elevation,landcover,biogeographicalRegion
0,2021-01-24,2021,1.0,24.0,Mixed woodland (with coniferous and deciduous ...,DK,Xylohypha ferruginosa (Corda) S.Hughes,Fungi,Ascomycota,Eurotiomycetes,...,3052832307,Sjælland,Næstved,0-3052832307.JPG,2421,wood,0,0.0,16.0,continental
1,2021-02-27,2021,2.0,27.0,garden,DK,"Comatricha alta Preuss, 1851",Protozoa,Mycetozoa,Myxomycetes,...,3061954303,Hovedstaden,Gribskov,0-3061954303.JPG,386,wood,0,0.0,17.0,continental
2,2021-02-27,2021,2.0,27.0,garden,DK,"Comatricha alta Preuss, 1851",Protozoa,Mycetozoa,Myxomycetes,...,3061954303,Hovedstaden,Gribskov,1-3061954303.JPG,386,wood,0,0.0,17.0,continental
3,2021-02-27,2021,2.0,27.0,garden,DK,"Comatricha alta Preuss, 1851",Protozoa,Mycetozoa,Myxomycetes,...,3061954303,Hovedstaden,Gribskov,2-3061954303.JPG,386,wood,0,0.0,17.0,continental
4,2021-02-27,2021,2.0,27.0,garden,DK,"Comatricha alta Preuss, 1851",Protozoa,Mycetozoa,Myxomycetes,...,3061954303,Hovedstaden,Gribskov,3-3061954303.JPG,386,wood,0,0.0,17.0,continental


In [3]:
validation_data = pd.read_csv("/kaggle/input/fungi-clef-2025/metadata/FungiTastic-FewShot/FungiTastic-FewShot-Val.csv")
test_data = pd.read_csv("/kaggle/input/fungi-clef-2025/metadata/FungiTastic-FewShot/FungiTastic-FewShot-Test.csv")

test_data.head()

,eventDate,year,month,day,habitat,countryCode,hasCoordinate,substrate,latitude,longitude,coorUncert,observationID,region,district,filename,metaSubstrate,elevation,landcover,biogeographicalRegion
0,2023-01-08,2023.0,1.0,8.0,Deciduous woodland,DK,True,dead wood (including bark),55.993269,12.290225,50.0,4100099350,Hovedstaden,Hillerød,0-4100099350.JPG,wood,43.0,12.0,continental
1,2023-01-10,2023.0,1.0,10.0,Unmanaged deciduous woodland,DK,True,dead wood (including bark),55.973423,10.550840,5.0,4100096393,Midtjylland,Samsø,1-4100096393.JPG,wood,64.0,12.0,continental
2,2023-01-09,2023.0,1.0,9.0,lawn,ES,True,soil,28.501376,-17.874959,5.0,4100103428,Islas Canarias,Santa Cruz de Tenerife,0-4100103428.JPG,jord,0.0,0.0,NaN
3,2023-01-09,2023.0,1.0,9.0,lawn,ES,True,soil,28.501376,-17.874959,5.0,4100103428,Islas Canarias,Santa Cruz de Tenerife,1-4100103428.JPG,jord,0.0,0.0,NaN
4,2023-01-09,2023.0,1.0,9.0,lawn,ES,True,soil,28.501376,-17.874959,5.0,4100103428,Islas Canarias,Santa Cruz de Tenerife,2-4100103428.JPG,jord,0.0,0.0,NaN


In [4]:
print("Length of train data before merger:", len(train_data))
train_data = pd.concat([train_data, validation_data], axis=0)
print("Length of train data after merger:", len(train_data))

Length of train data before merger: 7819
Length of train data after merger: 10104


In [5]:
numeric_cols = []
object_cols = []
for i in train_data.columns:
    if train_data[i].dtypes == "int64" or train_data[i].dtypes == "float64":
        numeric_cols.append(i)
    else:
        object_cols.append(i)
numeric_cols, object_cols

(['year',
  'month',
  'day',
  'latitude',
  'longitude',
  'coorUncert',
  'observationID',
  'category_id',
  'poisonous',
  'elevation',
  'landcover'],
 ['eventDate',
  'habitat',
  'countryCode',
  'scientificName',
  'kingdom',
  'phylum',
  'class',
  'order',
  'family',
  'genus',
  'specificEpithet',
  'hasCoordinate',
  'species',
  'iucnRedListCategory',
  'substrate',
  'region',
  'district',
  'filename',
  'metaSubstrate',
  'biogeographicalRegion'])

In [6]:
df_numeric = train_data.loc[:, numeric_cols]
df_numeric.head()

,year,month,day,latitude,longitude,coorUncert,observationID,category_id,poisonous,elevation,landcover
0,2021.0,1.0,24.0,55.227983,11.680016,25.0,3052832307,2421,0,0.0,16.0
1,2021.0,2.0,27.0,56.087201,12.215145,10.0,3061954303,386,0,0.0,17.0
2,2021.0,2.0,27.0,56.087201,12.215145,10.0,3061954303,386,0,0.0,17.0
3,2021.0,2.0,27.0,56.087201,12.215145,10.0,3061954303,386,0,0.0,17.0
4,2021.0,2.0,27.0,56.087201,12.215145,10.0,3061954303,386,0,0.0,17.0


In [7]:
object_df = train_data.loc[:, object_cols]
object_df.head()

,eventDate,habitat,countryCode,scientificName,kingdom,phylum,class,order,family,genus,specificEpithet,hasCoordinate,species,iucnRedListCategory,substrate,region,district,filename,metaSubstrate,biogeographicalRegion
0,2021-01-24,Mixed woodland (with coniferous and deciduous ...,DK,Xylohypha ferruginosa (Corda) S.Hughes,Fungi,Ascomycota,Eurotiomycetes,Chaetothyriales,NaN,Xylohypha,ferruginosa,True,Xylohypha ferruginosa,NE,dead wood (including bark),Sjælland,Næstved,0-3052832307.JPG,wood,continental
1,2021-02-27,garden,DK,"Comatricha alta Preuss, 1851",Protozoa,Mycetozoa,Myxomycetes,Stemonitidales,Stemonitidaceae,Comatricha,alta,True,Comatricha alta,NE,dead wood (including bark),Hovedstaden,Gribskov,0-3061954303.JPG,wood,continental
2,2021-02-27,garden,DK,"Comatricha alta Preuss, 1851",Protozoa,Mycetozoa,Myxomycetes,Stemonitidales,Stemonitidaceae,Comatricha,alta,True,Comatricha alta,NE,dead wood (including bark),Hovedstaden,Gribskov,1-3061954303.JPG,wood,continental
3,2021-02-27,garden,DK,"Comatricha alta Preuss, 1851",Protozoa,Mycetozoa,Myxomycetes,Stemonitidales,Stemonitidaceae,Comatricha,alta,True,Comatricha alta,NE,dead wood (including bark),Hovedstaden,Gribskov,2-3061954303.JPG,wood,continental
4,2021-02-27,garden,DK,"Comatricha alta Preuss, 1851",Protozoa,Mycetozoa,Myxomycetes,Stemonitidales,Stemonitidaceae,Comatricha,alta,True,Comatricha alta,NE,dead wood (including bark),Hovedstaden,Gribskov,3-3061954303.JPG,wood,continental


In [8]:
less_uniques_in_object_cols = []
for i in object_cols:
    print(i)
    u = len(train_data[i].unique())
    if u <= 20:
        print("Uniques: ", train_data[i].unique())
        less_uniques_in_object_cols.append(i)
    else:
        print("Number of uniques: ", u)
    print("++++++++++++++++\n")

eventDate
Number of uniques:  2360
++++++++++++++++

habitat
Number of uniques:  31
++++++++++++++++

countryCode
Number of uniques:  26
++++++++++++++++

scientificName
Number of uniques:  2427
++++++++++++++++

kingdom
Uniques:  ['Fungi' 'Protozoa' 'Chromista']
++++++++++++++++

phylum
Uniques:  ['Ascomycota' 'Mycetozoa' 'Oomycota' 'Basidiomycota' 'Amoebozoa'
 'Mucoromycota' 'Entomophthoromycota' 'Glomeromycota' 'Chytridiomycota'
 'Blastocladiomycota' 'Zoopagomycota']
++++++++++++++++

class
Number of uniques:  33
++++++++++++++++

order
Number of uniques:  124
++++++++++++++++

family
Number of uniques:  365
++++++++++++++++

genus
Number of uniques:  1014
++++++++++++++++

specificEpithet
Number of uniques:  2158
++++++++++++++++

hasCoordinate
Uniques:  [ True]
++++++++++++++++

species
Number of uniques:  2413
++++++++++++++++

iucnRedListCategory
Uniques:  ['NE' 'VU' 'NT' 'EN' 'LC']
++++++++++++++++

substrate
Number of uniques:  31
++++++++++++++++

region
Number of uniques:  8

In [9]:
less_uniques_in_object_cols.remove("hasCoordinate")
less_uniques_in_object_cols

['kingdom',
 'phylum',
 'iucnRedListCategory',
 'metaSubstrate',
 'biogeographicalRegion']

In [10]:
import os
images_dic_wrt_category_id = {}
iteration_train_data = train_data.loc[:, ["category_id", "filename"]]
train_image_folder_path = "/kaggle/input/fungi-clef-2025/images/FungiTastic-FewShot/train/300p"
val_image_folder_path = "/kaggle/input/fungi-clef-2025/images/FungiTastic-FewShot/val/300p"
train_image_paths = os.listdir(train_image_folder_path)
for idx, row in iteration_train_data.iterrows():
    if row["filename"] not in train_image_paths:
        filename = os.path.join(val_image_folder_path, row["filename"])
    else:
        filename = os.path.join(train_image_folder_path, row["filename"])

    if row["category_id"] not in images_dic_wrt_category_id.keys():
        images_dic_wrt_category_id[row["category_id"]] = [filename]
    else:
        images_dic_wrt_category_id[row["category_id"]].append(filename)

In [11]:
category_val_counts = train_data.loc[:, ["category_id"]].value_counts().reset_index()

In [12]:
class_to_augment = category_val_counts[category_val_counts["count"]<5]["category_id"].tolist()
class_to_augment[:5]

[237, 46, 1182, 885, 2124]

In [13]:
import os
import numpy as np
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

2025-10-30 04:30:44.916368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761798644.938636     124 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761798644.945391     124 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [14]:
num_augmented_image = 5
datagen = ImageDataGenerator(
    rotation_range=30, 
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

output_dir = "/kaggle/working/dataset"
os.makedirs(output_dir, exist_ok=True)

In [15]:
import cv2
for label, image_list in images_dic_wrt_category_id.items():
    output_path = os.path.join(output_dir, str(label))
    os.makedirs(output_path, exist_ok=True)
    if int(label) not in class_to_augment:
        a=0
        for image_path in image_list:
            img = cv2.imread(image_path)
            cv2.imwrite(os.path.join(output_path, os.path.basename(image_path)), img)
            a+=1
            if a==5:
                break

    else:
        number_of_images = len(image_list)
        for image_path in image_list:
            img = cv2.imread(image_path)
            img_array = np.expand_dims(img, axis=0)
            cv2.imwrite(os.path.join(output_path, os.path.basename(image_path)), img)
        i=number_of_images
        for batch in datagen.flow(img_array, batch_size=1, save_to_dir=output_path, save_prefix="aug", save_format="jpg"):
            i+=1
            if i>=5:
                break

In [16]:
!pip install ftfy regex tqdm git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-e1tzb63r
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-e1tzb63r
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [17]:
import os
import clip
import torch
from PIL import Image
from tqdm import tqdm
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model, preprocess = clip.load("ViT-B/32", device=device)

In [19]:
DATASET_PATH = "/kaggle/working/dataset"

# Dict to hold class prototypes
class_prototypes = {}

def get_embedding(image_path):
    try:
        image = preprocess(Image.open(image_path).convert("RGB")).unsqueeze(0).to(device)
        with torch.no_grad():
            embedding = model.encode_image(image)
        return embedding.squeeze().cpu().numpy()
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

In [20]:
for class_name in tqdm(os.listdir(DATASET_PATH), desc="Building prototypes"):
    class_dir = os.path.join(DATASET_PATH, class_name)
    if not os.path.isdir(class_dir): continue

    embeddings = []
    for image_file in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_file)
        emb = get_embedding(image_path)
        if emb is not None:
            embeddings.append(emb)

    if len(embeddings) >= 1:
        prototype = np.mean(embeddings, axis=0)
        class_prototypes[class_name] = prototype
    else:
        print(f"Skipping {class_name}: no valid embeddings")

print(f"Built prototypes for {len(class_prototypes)} classes.")

Building prototypes: 100%|██████████| 2427/2427 [03:02<00:00, 13.32it/s]

Built prototypes for 2427 classes.


In [21]:
a = 0
for i in os.listdir(DATASET_PATH):
    print(i)
    a+=1
    if a%5==0:
        break

141
853
1401
279
1866


In [22]:
def classify_image(test_image_path, top_k=10):
    test_emb = get_embedding(test_image_path).reshape(1, -1)
    all_classes = list(class_prototypes.keys())
    all_prototypes = np.vstack([class_prototypes[c] for c in all_classes])

    similarities = cosine_similarity(test_emb, all_prototypes)[0]
    top_indices = similarities.argsort()[::-1][:top_k]

    results = [(all_classes[i], similarities[i]) for i in top_indices]
    return results

In [23]:
test_path = os.path.join(DATASET_PATH, "130")
test_path = os.path.join(test_path, os.listdir(test_path)[0])

In [24]:
results = classify_image(test_path)

In [25]:
test_data.head()

,eventDate,year,month,day,habitat,countryCode,hasCoordinate,substrate,latitude,longitude,coorUncert,observationID,region,district,filename,metaSubstrate,elevation,landcover,biogeographicalRegion
0,2023-01-08,2023.0,1.0,8.0,Deciduous woodland,DK,True,dead wood (including bark),55.993269,12.290225,50.0,4100099350,Hovedstaden,Hillerød,0-4100099350.JPG,wood,43.0,12.0,continental
1,2023-01-10,2023.0,1.0,10.0,Unmanaged deciduous woodland,DK,True,dead wood (including bark),55.973423,10.550840,5.0,4100096393,Midtjylland,Samsø,1-4100096393.JPG,wood,64.0,12.0,continental
2,2023-01-09,2023.0,1.0,9.0,lawn,ES,True,soil,28.501376,-17.874959,5.0,4100103428,Islas Canarias,Santa Cruz de Tenerife,0-4100103428.JPG,jord,0.0,0.0,NaN
3,2023-01-09,2023.0,1.0,9.0,lawn,ES,True,soil,28.501376,-17.874959,5.0,4100103428,Islas Canarias,Santa Cruz de Tenerife,1-4100103428.JPG,jord,0.0,0.0,NaN
4,2023-01-09,2023.0,1.0,9.0,lawn,ES,True,soil,28.501376,-17.874959,5.0,4100103428,Islas Canarias,Santa Cruz de Tenerife,2-4100103428.JPG,jord,0.0,0.0,NaN


In [26]:
test_data_path = "/kaggle/input/fungi-clef-2025/images/FungiTastic-FewShot/test/300p"
result_dic = {}
for oID, filename in test_data.loc[:, ["observationID", "filename"]].values:
    test_image_path = os.path.join(test_data_path, filename)
    results = classify_image(test_image_path)
    r = []
    for res in results:
        r.append(res[0])
    top_10_class = " ".join(r)
    result_dic[oID] = top_10_class

In [27]:
result_dic

{4100099350: '1796 1882 1621 445 318 2158 1496 30 291 247',
 4100096393: '2404 885 342 360 59 356 1549 1481 1522 1360',
 4100103428: '770 16 1579 2008 45 769 1569 13 1813 773',
 4100096438: '1310 1911 654 1414 977 126 128 1477 1210 1622',
 4100102708: '675 613 928 1953 2426 352 382 2041 1478 1696',
 4100103826: '351 1598 1984 211 1523 1623 927 1231 1419 339',
 4100096876: '2393 1336 1797 293 1623 30 1419 1054 2045 1',
 4100104007: '1624 1696 264 928 2045 2038 606 608 668 1737',
 4100100146: '70 847 1076 1030 613 352 1398 1803 660 1536',
 4100104174: '1744 384 1301 608 1624 1696 816 1508 2355 1438',
 4847334709: '1518 1263 629 2163 1795 1976 2104 1468 339 746',
 4100100236: '1310 1106 1414 1660 308 816 1331 1723 128 1489',
 4100097236: '1624 1508 1737 2135 1637 1744 928 2125 1739 1536',
 4100107315: '1703 728 1357 2239 2377 360 59 1430 2325 1884',
 4100111460: '1625 1696 2038 29 928 914 72 1273 2041 11',
 4169760342: '98 1174 2091 1739 668 578 63 726 613 1334',
 4100111660: '138 1549 10

In [28]:
sub_df = pd.DataFrame(list(result_dic.items()), columns=["observationId", "predictions"])
sub_df.head()

,observationId,predictions
0,4100099350,1796 1882 1621 445 318 2158 1496 30 291 247
1,4100096393,2404 885 342 360 59 356 1549 1481 1522 1360
2,4100103428,770 16 1579 2008 45 769 1569 13 1813 773
3,4100096438,1310 1911 654 1414 977 126 128 1477 1210 1622
4,4100102708,675 613 928 1953 2426 352 382 2041 1478 1696


In [29]:
sub_df.to_csv("submission.csv", index=False)